## Model Selection

This notebook should include preliminary and baseline modeling.
- Try as many different models as possible.
- Don't worry about hyperparameter tuning or cross validation here.
- Ideas include:
    - linear regression
    - support vector machines
    - random forest
    - xgboost

In [ ]:
# import models and fit
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
#Setting up data for Train/Test Split

data = pd.read_csv('cleaned_income_merged_data.csv')

X = data.drop(columns = ['sold_price'], axis=1)
y = data['sold_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
print(X_train.shape)
print(X_test.shape)

(953, 127)
(239, 127)


In [16]:
# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Lasso Regression
lasso_model = Lasso(alpha=1000)
lasso_model.fit(X_train, y_train)

# Ridge Regression
ridge_model = Ridge(alpha=100)
ridge_model.fit(X_train, y_train)

# Support Vector Regression (SVR)
svr_model_rbf = SVR(kernel='rbf')
svr_model_rbf.fit(X_train, y_train)

# Support Vector Regression (SVR)
svr_model_linear = SVR(kernel='linear')
svr_model_linear.fit(X_train, y_train)

# Random Forest Regression
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(X_train, y_train)

# XGBoost Regression
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [29]:
# Polynomial Regression (using PolynomialFeatures)
poly = PolynomialFeatures(degree=2) #Unable to run higher than 4 due to number of columns slowing down system
X_train_poly = poly.fit_transform(X_train)
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)
y_train_pred = poly_model.predict(X_train_poly)
X_test_poly = poly.transform(X_test)
y_test_pred = poly_model.predict(X_test_poly)

# Evaluate model performance on training and testing sets
mae_train = mean_absolute_error(y_train, y_train_pred)
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)

mae_test = mean_absolute_error(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)

# Print results
print("Polynomial Regression Model - Training Metrics:")
print(f"Mean Absolute Error (Training): {mae_train}")
print(f"Mean Squared Error (Training): {mse_train}")
print(f"R-squared (Training): {r2_train}")
print(f"RMSE (Training): {rmse_train}\n")

print("Polynomial Regression Model - Testing Metrics:")
print(f"Mean Absolute Error (Testing): {mae_test}")
print(f"Mean Squared Error (Testing): {mse_test}")
print(f"R-squared (Testing): {r2_test}")
print(f"RMSE (Testing): {rmse_test}")

Polynomial Regression Model - Training Metrics:
Mean Absolute Error (Training): 0.012078611760674844
Mean Squared Error (Training): 0.0005062555029127422
R-squared (Training): 0.9999999999999984
RMSE (Training): 0.022500122286617515

Polynomial Regression Model - Testing Metrics:
Mean Absolute Error (Testing): 531774261.5374953
Mean Squared Error (Testing): 6.736467537342095e+19
R-squared (Testing): -553116018.8369081
RMSE (Testing): 8207598636.228562


In [23]:
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Hyperparameter grid 
param_dist = {
    'n_estimators': [50, 100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(rf_model, param_distributions=param_dist, n_iter=50, cv=5, scoring='neg_mean_squared_error', random_state=42, error_score=np.nan)
random_search.fit(X_train, y_train)

# Best parameters and score
print("Best Random Forest Model:", random_search.best_params_)
print("Best CV score (MSE):", -random_search.best_score_)

# Perform cross-validation on best model
cv_scores = cross_val_score(random_search.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f"Cross-validation MSE: {-cv_scores.mean()}")


c:\Users\lai29\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
65 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
65 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\lai29\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lai29\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "c:\U

Best Random Forest Model: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None}
Best CV score (MSE): 209027789747.7816
Cross-validation MSE: 209027789747.7816


In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge

# Define parameter grid
param_grid = {'alpha': [0.1, 1, 10, 100, 1000]}  # values to test for Alpha

# For Lasso regression
lasso = Lasso(max_iter=10000)
lasso_grid_search = GridSearchCV(lasso, param_grid, cv=5)
lasso_grid_search.fit(X_train, y_train)
print(f"Best alpha for Lasso: {lasso_grid_search.best_params_}")

# For Ridge regression
ridge = Ridge(max_iter=10000)
ridge_grid_search = GridSearchCV(ridge, param_grid, cv=5)
ridge_grid_search.fit(X_train, y_train)
print(f"Best alpha for Ridge: {ridge_grid_search.best_params_}")

c:\Users\lai29\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.150e+11, tolerance: 1.565e+10
  model = cd_fast.enet_coordinate_descent(
c:\Users\lai29\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.688e+12, tolerance: 2.735e+10
  model = cd_fast.enet_coordinate_descent(
c:\Users\lai29\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

Best alpha for Lasso: {'alpha': 1000}
Best alpha for Ridge: {'alpha': 100}


In [25]:
from sklearn.tree import DecisionTreeRegressor

# Initialize model
dt_model = DecisionTreeRegressor(random_state=10)
dt_model.fit(X_train, y_train)

# Make predictions and calculate metrics
y_pred = dt_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

# Print results
print("Decision Tree Regression:")
print(f"MAE: {mae}, MSE: {mse}, R2: {r2}, RMSE: {rmse}")

Decision Tree Regression:
MAE: 139946.2677824268, MSE: 62125030636.54393, R2: 0.48990551075246513, RMSE: 249248.93307002124


In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Initialize model
knn_model = KNeighborsRegressor()
knn_model.fit(X_train, y_train)

# Calculate initial Score
y_pred = knn_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

# Print results
print("K-Nearest Neighbors Regression:")
print(f"MAE: {mae}, MSE: {mse}, R2: {r2}, RMSE: {rmse}")

K-Nearest Neighbors Regression:
MAE: 149025.89456066943, MSE: 90247990750.90477, R2: 0.25899428497634003, RMSE: 300413.03359026345


In [21]:
# Combine model information
models = [lr_model, lasso_model, ridge_model, svr_model_rbf, svr_model_linear, rf_model, xgb_model, knn_model, dt_model]
model_names = ['Linear Regression', 'Lasso Regression', 'Ridge Regression','Support Vector Regression', 'Random Forest Regression', 'XGBoost Regression', 'Nearest Neighbour', 'Decision Tree']
model_scores = []

# model scores
for model, name in zip(models, model_names):
    y_pred = model.predict(X_test)            
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mse)  # Calculate RMSE
    model_scores.append((name, mae, mse, r2, rmse))

# Print results for each model
for name, mae, mse, r2, rmse in model_scores:
    print(f"Model: {name}")
    print(f"Mean Absolute Error: {mae}")
    print(f"Mean Squared Error: {mse}")
    print(f"R-squared: {r2}")
    print(f"RMSE: {rmse}")
    print() #Empty line between 

Model: Linear Regression
Mean Absolute Error: 235101.23331020912
Mean Squared Error: 446373810556.0938
R-squared: -2.6650737806662805
RMSE: 668112.1242397071

Model: Lasso Regression
Mean Absolute Error: 223730.01977674325
Mean Squared Error: 639376577469.5048
R-squared: -4.249775579656559
RMSE: 799610.2659855642

Model: Ridge Regression
Mean Absolute Error: 235608.2928330447
Mean Squared Error: 2060006997961.7761
R-squared: -15.914248680523734
RMSE: 1435272.447294163

Model: Support Vector Regression
Mean Absolute Error: 189307.61745131513
Mean Squared Error: 124138454136.6094
R-squared: -0.01927259769501144
RMSE: 352332.874050392

Model: Random Forest Regression
Mean Absolute Error: 184993.7399862615
Mean Squared Error: 836540627162.7847
R-squared: -5.868644724601646
RMSE: 914625.9493163228

Model: XGBoost Regression
Mean Absolute Error: 96477.41887029288
Mean Squared Error: 47639403770.15816
R-squared: 0.6088436965711286
RMSE: 218264.5270541188

Model: Nearest Neighbour
Mean Absolut

Consider what metrics you want to use to evaluate success.
- If you think about mean squared error, can we actually relate to the amount of error?
- Try root mean squared error so that error is closer to the original units (dollars)
- What does RMSE do to outliers?
- Is mean absolute error a good metric for this problem?
- What about R^2? Adjusted R^2?
- Briefly describe your reasons for picking the metrics you use

In [26]:
# gather evaluation metrics and compare results

## Feature Selection - STRETCH

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its recommended you complete it if you have time!**

Even with all the preprocessing we did in Notebook 1, you probably still have a lot of features. Are they all important for prediction?

Investigate some feature selection algorithms (Lasso, RFE, Forward/Backward Selection)
- Perform feature selection to get a reduced subset of your original features
- Refit your models with this reduced dimensionality - how does performance change on your chosen metrics?
- Based on this, should you include feature selection in your final pipeline? Explain

Remember, feature selection often doesn't directly improve performance, but if performance remains the same, a simpler model is often preferrable. 



In [ ]:
# perform feature selection 
# refit models
# gather evaluation metrics and compare to the previous step (full feature set)